In [ ]:
pip install wikipedia-api

In [4]:
import wikipediaapi


wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

In [ ]:
from random import shuffle

def url_categorymembers(categorymembers, store, level=0, max_level=1):
    pages_cat = list(categorymembers.values())
    shuffle(pages_cat)
    i = 0 # nb de pages
    for c in pages_cat:
        if i==1000: #1000 pages par catégorie
            break
        if c.title[:6]!="Portal" and c.title[:8]!="Category":
            i+=1
            store.append(c.fullurl)
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            url_categorymembers(c.categorymembers, store, level=level + 1, max_level=max_level)
    return store




cat_arts = wiki_wiki.page("Category:The arts") # Arts
arts = url_categorymembers(cat_arts.categorymembers,[])

cat_games = wiki_wiki.page("Category:Games") # Games
games = url_categorymembers(cat_games.categorymembers,[])

cat_youth = wiki_wiki.page("Category:Youth") # Kids and Teens (not exact)
youth = url_categorymembers(cat_youth.categorymembers,[])

cat_reference = wiki_wiki.page("Category:Reference") # Reference
reference = url_categorymembers(cat_reference.categorymembers,[])

cat_shopping = wiki_wiki.page("Category:Shopping (activity)") # Shopping
shopping = url_categorymembers(cat_shopping.categorymembers,[])

cat_business = wiki_wiki.page("Category:Business") # Business
business = url_categorymembers(cat_business.categorymembers,[])

cat_health = wiki_wiki.page("Category:Health") # Health
health = url_categorymembers(cat_health.categorymembers,[])

cat_news = wiki_wiki.page("Category:News") # News
news = url_categorymembers(cat_news.categorymembers,[])

cat_geography = wiki_wiki.page("Category:Geography") # Regional (not exact)
geography = url_categorymembers(cat_geography.categorymembers,[])

cat_society = wiki_wiki.page("Category:Society") # Society
society = url_categorymembers(cat_society.categorymembers,[])

cat_computers = wiki_wiki.page("Category:Computers") # Computers
computers = url_categorymembers(cat_computers.categorymembers,[])

cat_home = wiki_wiki.page("Category:Home") # Home
home = url_categorymembers(cat_home.categorymembers,[])

cat_recreation = wiki_wiki.page("Category:Recreation") # Recreation
recreation = url_categorymembers(cat_recreation.categorymembers,[])

cat_science = wiki_wiki.page("Category:Science") # Science
science = url_categorymembers(cat_science.categorymembers,[])

cat_sports = wiki_wiki.page("Category:Sports") # Sports
sports = url_categorymembers(cat_sports.categorymembers,[])

cat_world = wiki_wiki.page("Category:World") # World
world = url_categorymembers(cat_world.categorymembers,[])